# 🔬 The Lilliput Effect: Climate vs. Taphonomy
**Project:** Deep-Time Dynamics
**Objective:** To determine if the observed shrinking of marine invertebrates during hyperthermal events (the "Lilliput Effect") is a biological response to temperature or a geological artifact of preservation bias.

### 📊 Data Sources
* **Fossils:** Paleobiology Database (PBDB) - Occurrences of marine invertebrates.
* **Climate:** CENOGRID (Westerhold et al., 2020) - Global Benthic $\delta^{18}O$ stack.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Loading Data...")

# 1. Load Fossil Context (Time & Rocks)
# Note: 'pbdb_data.csv' headers usually start around line 19
try:
    df_context = pd.read_csv('pbdb_data.csv', skiprows=19)
except FileNotFoundError:
    print("Error: 'pbdb_data.csv' not found. Please upload the file.")
    # Exit or handle error appropriately in a real notebook
    # For now, create an empty dataframe to avoid further errors
    df_context = pd.DataFrame()

# 2. Load Fossil Measurement File (Sizes)
# Note: 'pbdb_data (1).csv' headers usually start around line 17
try:
    df_meas = pd.read_csv('pbdb_data (1).csv', skiprows=17)
except FileNotFoundError:
    print("Error: 'pbdb_data (1).csv' not found. Please upload the file.")
    df_meas = pd.DataFrame()

# 3. Merge them together (The Magic Step)
# This matches the ID number from both files
if not df_context.empty and not df_meas.empty:
    df_final = pd.merge(df_context, df_meas, on='specimen_no', how='inner')
    print("Success! We have merged Time, Lithology, and Size.")
    print(df_final[['genus', 'max_ma', 'lithology1', 'measurement_type', 'average']].head())
else:
    df_final = pd.DataFrame()
    print("Skipping merge due to missing input files.")

# --- 2. Load Your Uploaded Climate Data ---
print("\nLoading Climate Data (TableS33.tab)...")

# The file uses tabs (\t) and has 91 lines of metadata before the table starts.
# We skip the metadata and use the standard CENOGRID columns.
try:
    df_climate = pd.read_csv(
        'TableS33.tab',
        sep='\t',
        skiprows=91  # Skips the text header to get to the real data
    )

    # Rename columns to match our project standard
    # 'Tuned time [Ma]' -> age_ma
    # 'Foram benth δ18O [‰ PDB] (VPDB CorrAdjusted)' -> d18o (Temperature proxy)
    # 'Foram benth δ13C [‰ PDB] (VPDB CorrAdjusted)' -> d13c (Carbon proxy)
    df_climate = df_climate.rename(columns={
        'Tuned time [Ma]': 'age_ma',
        'Foram benth δ18O [‰ PDB] (VPDB CorrAdjusted)': 'd18o',
        'Foram benth δ13C [‰ PDB] (VPDB CorrAdjusted)': 'd13c'
    })

    # Filter for Cenozoic only (0-66 Ma)
    df_climate = df_climate[(df_climate['age_ma'] >= 0) & (df_climate['age_ma'] <= 66)]

    print(f"   -> Success! Loaded {len(df_climate)} climate data points.")
    print(f"   -> Time Range: {df_climate['age_ma'].min():.2f} to {df_climate['age_ma'].max():.2f} Ma")

except FileNotFoundError:
    print("Error: 'TableS33.tab' not found. Please upload the file.")
    df_climate = pd.DataFrame()
except Exception as e:
    print(f"   -> Error loading climate file: {e}")

# --- 3. The Binning Process (Aligning Fossils to Climate) ---
# Ensure you have 'df_final' (your fossil data) from the previous step.
if 'df_final' in locals() and not df_final.empty and 'df_climate' in locals() and not df_climate.empty:
    print("\nAligning Fossils with Climate...")

    # A. Calculate Fossil Midpoint Age
    if 'min_ma' in df_final.columns:
        df_final['age_mid'] = (df_final['max_ma'] + df_final['min_ma']) / 2
    else:
        df_final['age_mid'] = df_final['max_ma']

    # B. Define Time Bins (0.1 Ma slices)
    # This creates buckets like 45.0, 45.1, 45.2 Ma
    bins = np.arange(0, 66.1, 0.1)
    labels = bins[:-1] + 0.05

    # C. Assign Fossils to Time Bins
    df_final['time_bin'] = pd.cut(df_final['age_mid'], bins=bins, labels=labels, right=False)

    # D. Bin climate data and merge d18o into df_final
    df_climate['time_bin'] = pd.cut(df_climate['age_ma'], bins=bins, labels=labels, right=False)
    # Taking the mean d18o for each bin
    df_climate_binned = df_climate.groupby('time_bin', observed=False)['d18o'].mean().reset_index()

    # Merge climate data (d18o) into the fossil dataframe
    # Using a left merge to keep all fossils, and add d18o where time_bins match
    df_final = pd.merge(df_final, df_climate_binned, on='time_bin', how='left')

    print(f"   -> Successfully merged climate data (d18o) into df_final.")
else:
    print("Skipping alignment of fossils with climate due to missing data.")



## 🛠️ Feature Engineering: Preparing Data for AI
This section transforms the raw merged data into a structured format suitable for machine learning algorithms. We extract relevant features such as absolute latitude and one-hot encode lithology types to represent rock compositions numerically.

---

In [ ]:
# --- PHASE 2: FEATURE ENGINEERING ---
# This converts your raw data into a matrix of numbers for the AI.

# 1. Create a fresh copy for Machine Learning
if 'df_final' in locals() and not df_final.empty:
    df_ml = df_final.copy()

    # 2. Filter for Realism (Remove bad data)
    # We need rows that have both a Body Size (average) and a Temperature (d18o)
    df_ml = df_ml.dropna(subset=['average', 'd18o'])
    df_ml = df_ml[df_ml['average'] > 0] # Size must be positive

    # 3. Geography: Calculate Absolute Latitude
    # We care about "Distance from Equator" (0 to 90), not North/South (-90 to +90).
    # PBDB usually stores this in 'lat' or 'paleolat'.
    if 'paleolat' in df_ml.columns:
        df_ml['latitude_abs'] = df_ml['paleolat'].abs()
    elif 'lat' in df_ml.columns:
        df_ml['latitude_abs'] = df_ml['lat'].abs()
    else:
        print("Warning: Latitude column missing. Using 0.")
        df_ml['latitude_abs'] = 0

    # 4. Taphonomy: One-Hot Encoding (The "Rock Type" Translation)
    # This converts "Sandstone" into [1, 0, 0] and "Shale" into [0, 1, 0]

    # A. Clean the text (lowercase, remove punctuation)
    df_ml['lith_clean'] = df_ml['lithology1'].astype(str).str.lower().str.replace('"', '').str.strip()

    # B. Simplify: Only keep the Top 10 most common rocks (Group others as "other")
    top_10_rocks = df_ml['lith_clean'].value_counts().nlargest(10).index
    df_ml['lith_simple'] = df_ml['lith_clean'].apply(lambda x: x if x in top_10_rocks else 'other')

    # C. Convert to Numbers (Get Dummies)
    lith_dummies = pd.get_dummies(df_ml['lith_simple'], prefix='lith')
    df_ml = pd.concat([df_ml, lith_dummies], axis=1)

    # --- 5. Final Check ---
    print("Feature Engineering Complete!")
    print(f"Final Dataset Size: {len(df_ml)} rows")
    print("Columns ready for AI:", ['d18o', 'latitude_abs'] + list(lith_dummies.columns))

    # Show the 'Translated' Data
    print(df_ml[['genus', 'average', 'd18o', 'lith_sandstone', 'lith_shale']].head(5))
else:
    df_ml = pd.DataFrame()
    print("Skipping Feature Engineering due to missing or empty df_final.")



## 🤖 Machine Learning: Random Forest Analysis
Here, we employ a Random Forest Regressor to predict fossil body size based on the engineered features. The primary goal is to identify which factors (climate, geography, or lithology) have the most significant influence on body size variation.

---

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

if 'df_ml' in locals() and not df_ml.empty:
    # --- 1. Define Predictors (X) and Target (y) ---
    # Target: The average body size of the animal
    y = df_ml['average']

    # Predictors: Temperature (d18o), Geography (latitude), and Rock Type (lith_*)
    # Explicitly select only the numerical and one-hot encoded features
    predictor_columns = [
        'd18o',
        'latitude_abs',
    ]

    # Dynamically add lithology columns if they exist
    lithology_cols = [col for col in df_ml.columns if col.startswith('lith_')]
    predictor_columns.extend(lithology_cols)

    X = df_ml[predictor_columns]

    # (Optional) Verify we only have numbers
    print("Predictor Columns:", X.columns.tolist())

    # --- 2. Split Data (Train vs. Test) ---
    # We keep 20% of the data hidden to test the model's accuracy later
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # --- 3. Train the Random Forest ---
    print("Training the model... (This might take a moment)")
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # --- 4. Evaluate Performance ---
    # How well can we predict body size? (1.0 is perfect, 0.0 is random guessing)
    score = model.score(X_test, y_test)
    print(f"Model Accuracy (R^2 Score): {score:.4f}")

    # --- 5. THE RESULTS (Feature Importance) ---
    # This is the answer to your Research Question.
    importances = pd.DataFrame({
        'Feature': X.columns,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)

    print("\n--- RESEARCH RESULTS: What drives Body Size? ---")
    print(importances)

    # --- 6. Interpretation Helper ---
    top_feature = importances.iloc[0]['Feature']
    print(f"\nCONCLUSION: The most dominant factor driving body size is '{top_feature}'.")

    if 'd18o' in top_feature:
        print("-> RESULT: ECOLOGICAL SIGNAL DETECTED. Climate (Temperature) is the main driver.")
        print("   (This supports the 'Lilliput Effect' hypothesis).")
    elif 'latitude_abs' in top_feature:
        print("-> RESULT: GEOGRAPHIC SIGNAL DETECTED. Latitude is the main driver.")
        print("   (This could relate to latitudinal diversity gradients or climate zones).")
    else:
        print("-> RESULT: TAPHONOMIC/OTHER SIGNAL DETECTED. A lithology or other factor is the main driver.")
        print("   (This might indicate preservation bias or other environmental controls).")
else:
    print("Skipping Random Forest Analysis due to missing or empty df_ml.")


## 📈 Visualizing the Lilliput Effect: Body Size vs. Climate
This plot illustrates the relationship between mean body size and global temperature (inferred from the d18O proxy) over the Cenozoic Era. This helps us visually assess how body size responds to climatic shifts and whether the 'Lilliput Effect' is evident.

---

In [ ]:
if 'df_ml' in locals() and not df_ml.empty:
    # --- 1. Prepare Data for Plotting ---
    # Group by Time Bin to get the global average size per 0.1 Ma
    # This removes the "noise" of individual fossils and shows the global trend
    plot_data = df_ml.groupby('time_bin', observed=False)[['average', 'd18o']].mean().reset_index()

    # Convert time_bin back to numeric Age (midpoint)
    plot_data['age_ma'] = plot_data['time_bin']

    # Drop empty bins
    plot_data = plot_data.dropna()

    # Ensure plot_data is not empty after dropping NaNs
    if not plot_data.empty:
        # --- 2. Create the Dual-Axis Plot ---
        fig, ax1 = plt.subplots(figsize=(12, 6))

        # A. Plot Body Size (The Target)
        color_size = 'tab:blue'
        ax1.set_xlabel('Time (Million Years Ago)')
        ax1.set_ylabel('Mean Body Size (mm)', color=color_size, fontsize=12, fontweight='bold')
        ax1.plot(plot_data['age_ma'], plot_data['average'], color=color_size, linewidth=2, label='Body Size')
        ax1.tick_params(axis='y', labelcolor=color_size)
        ax1.invert_xaxis() # Geologists always plot time from Right (Old) to Left (Young)

        # B. Plot Temperature (The Predictor)
        # We use a second Y-axis (right side)
        ax2 = ax1.twinx()
        color_temp = 'tab:red'
        ax2.set_ylabel('Global Temperature Proxy (d18O)', color=color_temp, fontsize=12, fontweight='bold')
        # Note: Higher d18O means COLDER. We invert this axis so UP = WARMER
        ax2.plot(plot_data['age_ma'], plot_data['d18o'], color=color_temp, linestyle='--', alpha=0.6, label='Temperature (d18O)')
        ax2.tick_params(axis='y', labelcolor=color_temp)
        ax2.invert_yaxis()

        # --- 3. Formatting ---
        plt.title('The Lilliput Effect: Global Body Size vs. Climate (Cenozoic Era)', fontsize=14)
        ax1.grid(True, alpha=0.3)
        fig.tight_layout()
        plt.show()
    else:
        print("Plotting skipped: plot_data is empty after cleaning.")
else:
    print("Skipping Visualization of Lilliput Effect due to missing or empty df_ml.")


## 🧠 Neural ODEs: Learning Continuous Dynamics
This section demonstrates the use of Neural Ordinary Differential Equations (Neural ODEs) to learn continuous dynamic systems from irregularly sampled time-series data. This technique is particularly valuable for paleontological records, which are often sparse and gappy.

---

In [ ]:
# --- STEP 1: INSTALLATION ---
# We need the torchdiffeq library for the ODE solvers
!pip install torchdiffeq


In [ ]:
import os
import argparse
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# Import the ODE solver
from torchdiffeq import odeint

# --- STEP 2: GENERATE SYNTHETIC PALEO DATA (SINE WAVE WITH GAPS) ---
# We simulate a "perfect" fossil record (sine wave) and then "erode" it
# to create irregular sampling, mimicking real geological data.

data_size = 1000
batch_time = 20
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Running on device: {device}")

# True Dynamics (The "Laws of Physics" we want to learn)
true_y0 = torch.tensor([[2., 0.]]).to(device) # Initial condition (Diversity, Rate)
t = torch.linspace(0., 25., data_size).to(device) # Time (0 to 25 Ma)
true_A = torch.tensor([[-0.1, 2.0], [-2.0, -0.1]]).to(device) # Spiral dynamics Matrix

class Lambda(nn.Module):
    def forward(self, t, y):
        return torch.mm(y**3, true_A) # Cubic dynamics (non-linear)

# Generate the "True" history (The ground truth)
with torch.no_grad():
    true_y = odeint(Lambda(), true_y0, t, method='dopri5')

def get_batch():
    # This function creates "Irregular Gaps"
    # We grab random slices of time, simulating imperfect preservation
    s = torch.from_numpy(np.random.choice(np.arange(data_size - batch_time, dtype=np.int64), batch_size, replace=False))
    batch_y0 = true_y[s]  # (M, D)
    batch_t = t[:batch_time]  # (T)
    batch_y = torch.stack([true_y[s + i] for i in range(batch_time)], dim=0)  # (T, M, D)
    return batch_y0.to(device), batch_t.to(device), batch_y.to(device)

# --- STEP 3: DEFINE THE NEURAL ODE (The "ODEFunc") ---
# Instead of predicting Y directly, we predict the DERIVATIVE (dy/dt).
# The Solver then integrates this to find Y.

class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        # A simple MLP (Multi-Layer Perceptron) that approximates the derivative
        self.net = nn.Sequential(
            nn.Linear(2, 50),
            nn.Tanh(),
            nn.Linear(50, 2), # Output is 2D (Diversity, Rate)
        )

        # Initialize weights for better convergence
        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        return self.net(y)

# --- STEP 4: TRAINING LOOP ---
func = ODEFunc().to(device)
optimizer = optim.RMSprop(func.parameters(), lr=1e-3)

print("Training Neural ODE... (Learning the underlying dynamics)")
start_time = time.time()

loss_history = []

for itr in range(1, 1001): # 1000 Iterations
    optimizer.zero_grad()
    batch_y0, batch_t, batch_y = get_batch()

    # FORWARD PASS: Integrate the Neural ODE from t0 to t_end
    # This uses the 'dopri5' solver (Runge-Kutta)
    pred_y = odeint(func, batch_y0, batch_t).to(device)

    # Calculate Loss (Difference between predicted trajectory and true data)
    loss = torch.mean(torch.abs(pred_y - batch_y))
    loss.backward()
    optimizer.step()

    loss_history.append(loss.item())

    if itr % 100 == 0:
        print(f"Iter {itr:04d} | Total Loss {loss.item():.6f}")

print(f"Training Complete in {time.time() - start_time:.2f}s")

# --- STEP 5: VISUALIZATION (Verify Solver) ---
# We verify if the model learned the "Sine Wave" shape despite the gaps.

with torch.no_grad():
    pred_y = odeint(func, true_y0, t)

# Visualize the synthetic data and the learned trajectory
plt.figure(figsize=(10, 6))
plt.plot(t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 0], label='True Y (Dimension 0)')
plt.plot(t.cpu().numpy(), pred_y.cpu().numpy()[:, 0, 0], '--', label='Predicted Y (Dimension 0)')
plt.scatter(t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 0], s=5, alpha=0.5, label='Sampled Data')
plt.title('Neural ODE: Learning Synthetic Dynamics')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()



## 🔮 Time-Series Forecasting: Chronos-T5 for Biodiversity
This section explores the application of Chronos-T5, a transformer-based foundation model for time-series forecasting, to predict future biodiversity trends based on the Cenozoic fossil record.

---

In [ ]:
import sys
!{sys.executable} -m pip install chronos-forecasting --force-reinstall

print("Chronos-forecasting re-installation initiated. This may take a moment.")


In [ ]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from chronos import ChronosPipeline

# --- 1. RELOAD DATA (In case variables were lost during restart) ---
print("1. Preparing Fossil Time Series...")
try:
    # Try loading processed data first
    df_ml = pd.read_csv('Lilliput_Project_Data.csv')
except FileNotFoundError:
    print("Warning: 'Lilliput_Project_Data.csv' not found. Attempting to re-process data.")
    # Fallback: Re-create from raw files
    # This assumes raw files (pbdb_data.csv, pbdb_data (1).csv, TableS33.tab) are available
    try:
        df_context = pd.read_csv('pbdb_data.csv', skiprows=19)
        df_meas = pd.read_csv('pbdb_data (1).csv', skiprows=17)
        df_final = pd.merge(df_context, df_meas, on='specimen_no', how='inner')

        df_climate = pd.read_csv(
            'TableS33.tab',
            sep='\t',
            skiprows=91
        )
        df_climate = df_climate.rename(columns={
            'Tuned time [Ma]': 'age_ma',
            'Foram benth δ18O [‰ PDB] (VPDB CorrAdjusted)': 'd18o',
            'Foram benth δ13C [‰ PDB] (VPDB CorrAdjusted)': 'd13c'
        })
        df_climate = df_climate[(df_climate['age_ma'] >= 0) & (df_climate['age_ma'] <= 66)]

        # Create midpoint age
        if 'min_ma' in df_final.columns:
            df_final['age_mid'] = (df_final['max_ma'] + df_final['min_ma']) / 2
        else:
            df_final['age_mid'] = df_final['max_ma']

        # Define Time Bins (0.1 Ma slices)
        bins = np.arange(0, 66.1, 0.1)
        labels = bins[:-1] + 0.05
        df_final['time_bin'] = pd.cut(df_final['age_mid'], bins=bins, labels=labels, right=False)
        df_climate['time_bin'] = pd.cut(df_climate['age_ma'], bins=bins, labels=labels, right=False)
        df_climate_binned = df_climate.groupby('time_bin', observed=False)['d18o'].mean().reset_index()
        df_final = pd.merge(df_final, df_climate_binned, on='time_bin', how='left')

        # Redo Feature Engineering steps to get df_ml
        df_ml = df_final.copy()
        df_ml = df_ml.dropna(subset=['average', 'd18o'])
        df_ml = df_ml[df_ml['average'] > 0]
        if 'paleolat' in df_ml.columns:
            df_ml['latitude_abs'] = df_ml['paleolat'].abs()
        elif 'lat' in df_ml.columns:
            df_ml['latitude_abs'] = df_ml['lat'].abs()
        else:
            df_ml['latitude_abs'] = 0
        df_ml['lith_clean'] = df_ml['lithology1'].astype(str).str.lower().str.replace('"', '').str.strip()
        top_10_rocks = df_ml['lith_clean'].value_counts().nlargest(10).index
        df_ml['lith_simple'] = df_ml['lith_clean'].apply(lambda x: x if x in top_10_rocks else 'other')
        lith_dummies = pd.get_dummies(df_ml['lith_simple'], prefix='lith')
        df_ml = pd.concat([df_ml, lith_dummies], axis=1)

    except Exception as e:
        print(f"Error during data reprocessing for Chronos: {e}")
        df_ml = pd.DataFrame() # Ensure df_ml is empty if reprocessing fails

if 'df_ml' in locals() and not df_ml.empty:
    # Create Time Series: Count unique genera per time step
    # Sort descending (Oldest -> Youngest) because Chronos expects time to move forward
    ts_data = df_ml.groupby('age_mid')['genus'].nunique().sort_index(ascending=False)

    # --- 2. LOAD THE AI MODEL ---
    print("2. Loading Chronos-T5 (Foundation Model)...")
    pipeline = ChronosPipeline.from_pretrained(
        "amazon/chronos-t5-small",
        device_map="cuda" if torch.cuda.is_available() else "cpu",
        torch_dtype=torch.bfloat16,
    )

    # --- 3. RUN FORECAST ---
    print("3. Forecasting the Future...")
    # Convert data to tensor
    context = torch.tensor(ts_data.values)
    prediction_length = 20 # Forecast 20 steps into the "future"

    forecast = pipeline.predict(
        context,
        prediction_length,
        num_samples=20,
    )

    # --- 4. VISUALIZE RESULTS ---
    low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)
    history_len = len(context)

    plt.figure(figsize=(12, 6))
    # Plot History (Real Fossil Record)
    plt.plot(np.arange(history_len), context, color="black", label="History (Fossil Record)")
    # Plot Forecast (AI Prediction)
    plt.plot(np.arange(history_len, history_len + prediction_length), median, color="blue", label="AI Forecast")
    # Plot Uncertainty (Confidence Interval)
    plt.fill_between(
        np.arange(history_len, history_len + prediction_length),
        low, high, color="blue", alpha=0.3, label="10-90% Quantile"
    )

    plt.title("Biodiversity Forecasting with Chronos-T5", fontsize=14)
    plt.xlabel("Time Steps")
    plt.ylabel("Number of Unique Genera")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Skipping Chronos-T5 initial forecasting due to missing or empty df_ml.")


In [ ]:
import sys
!{sys.executable} -m pip install datasets

print("Datasets library installation initiated.")


In [ ]:
import torch
from datasets import Dataset

if 'ts_data' in locals() and not ts_data.empty:
    # 1. Convert ts_data (pandas Series) into a NumPy array.
    full_series_np = ts_data.values

    # 2. Define the length of the validation set (val_len)
    val_len = 9 # The instructions suggest 9 points for ~20% of total data.

    # 3. Split the full time series NumPy array into train_series and val_series.
    train_series_np = full_series_np[:-val_len]
    val_series_np = full_series_np[-val_len:]

    print(f"Total data points: {len(full_series_np)}")
    print(f"Training series length: {len(train_series_np)}")
    print(f"Validation series length: {len(val_series_np)}")

    # 4. Define context_window and prediction_window
    context_window = 20 # Example: Observe 20 past data points
    prediction_window = 5 # Example: Predict 5 future data points

    # 5. Create a list of training samples.
    train_samples = []
    # Ensure there's enough data for at least one full sample
    if len(train_series_np) >= context_window + prediction_window:
        for i in range(len(train_series_np) - context_window - prediction_window + 1):
            context = torch.tensor(train_series_np[i : i + context_window], dtype=torch.float32)
            target = torch.tensor(train_series_np[i + context_window : i + context_window + prediction_window], dtype=torch.float32)
            train_samples.append({'context': context, 'target': target})

    print(f"Generated {len(train_samples)} training samples.")

    # 6. Create a single validation sample.
    # The context for this sample should be the entire train_series_np
    # The target should be the entire val_series_np
    val_context_tensor = torch.tensor(train_series_np, dtype=torch.float32)
    val_target_tensor = torch.tensor(val_series_np, dtype=torch.float32)
    val_sample = {'context': val_context_tensor, 'target': val_target_tensor}

    # 7. Convert the list of training samples into a datasets.Dataset object.
    train_dataset = Dataset.from_list(train_samples)

    print("Data preparation for fine-tuning complete.")
    print(f"Training Dataset: {train_dataset}")
    print(f"Validation Sample Context Shape: {val_sample['context'].shape}")
    print(f"Validation Sample Target Shape: {val_sample['target'].shape}")
else:
    print("Skipping Chronos data preparation due to missing or empty ts_data.")


In [ ]:
import torch
from chronos import ChronosPipeline

# 1. Instantiate the ChronosPipeline model
print("Loading Chronos-T5 model for fine-tuning...")
pipeline_ft = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)

# 2. Set the model to training mode
pipeline_ft.model.train()
print("Model set to training mode.")

# 3. Define the prediction_length for training
prediction_length_ft = prediction_window # prediction_window should be defined in the previous cell
print(f"Prediction length for fine-tuning set to: {prediction_length_ft}")


In [ ]:
import torch
from torch.optim import AdamW
import numpy as np
from tqdm.auto import tqdm

if 'pipeline_ft' in locals() and 'train_series_np' in locals() and not train_series_np.empty:
    # Access the inner Hugging Face T5 model for training
    inner_model = pipeline_ft.model.model
    inner_model.train()

    optimizer = AdamW(inner_model.parameters(), lr=1e-5)
    num_epochs = 20

    # Explicitly set the tokenizer's prediction_length to match the prediction_window
    pipeline_ft.tokenizer.config.prediction_length = prediction_window

    # Ensure train_data is correctly referenced (it should be train_series_np)
    train_data_tensor = torch.tensor(train_series_np, dtype=torch.float32)

    print("Starting fine-tuning...")

    for epoch in range(num_epochs):
        epoch_loss = 0 # To calculate average loss per epoch

        # Adjust max_start_idx to account for both context and prediction windows
        max_start_idx = len(train_data_tensor) - context_window - prediction_window
        if max_start_idx < 1:
            print(f"Skipping epoch {epoch+1} due to insufficient data for context_window {context_window} and prediction_window {prediction_window}")
            # Define avg_loss here to prevent NameError if epoch is skipped
            avg_loss = 0.0 # Or some appropriate default
            print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.4f}")
            continue

        for _ in tqdm(range(max_start_idx), desc=f"Epoch {epoch + 1}/{num_epochs}"):
            optimizer.zero_grad()

            # 1. Sample a random window
            # The start_idx now refers to the beginning of the context window
            start_idx = np.random.randint(0, max_start_idx)
            window_context = train_data_tensor[start_idx : start_idx + context_window]
            window_target = train_data_tensor[start_idx + context_window : start_idx + context_window + prediction_window]

            # Add a batch dimension (unsqueeze(0)) for both window and target
            window_context = window_context.unsqueeze(0)
            window_target = window_target.unsqueeze(0)

            # 2. Tokenize using the pipeline's tokenizer
            input_ids, attention_mask, scale = pipeline_ft.tokenizer.context_input_transform(window_context)

            # label_input_transform uses the SAME scale to tokenize the target
            # The length of window_target MUST be equal to prediction_length, which is prediction_window
            # Adjusted to unpack only 2 values as per ValueError
            labels, _ = pipeline_ft.tokenizer.label_input_transform(window_target, scale)

            # Move to GPU/CPU
            device = inner_model.device
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            # 3. Forward Pass & Update using the INNER MODEL
            outputs = inner_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / max_start_idx
        print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.4f}")

    print("Fine-tuning complete!")
else:
    print("Skipping Chronos fine-tuning due to missing pipeline_ft or empty train_series_np.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import mean_squared_error

if 'ts_data' in locals() and not ts_data.empty and 'pipeline_ft' in locals():
    # --- 1. SETUP VALIDATION DATA ---
    # We split the data again: 90% History (Context) vs 10% Future (Truth)
    split_idx = int(len(ts_data) * 0.9)
    train_data = torch.tensor(ts_data.values[:split_idx], dtype=torch.float32) # The Past
    valid_data = torch.tensor(ts_data.values[split_idx:], dtype=torch.float32) # The "Hidden" Future

    print(f"Forecasting the last {len(valid_data)} time steps (The 'Hidden' Future)...")

    # --- 2. SWITCH TO INFERENCE MODE ---
    pipeline_ft.model.eval() # Stop updating weights

    # --- 3. GENERATE PREDICTION ---
    # We feed it the History (train_data) and ask for the Future
    forecast = pipeline_ft.predict(
        train_data,
        len(valid_data),
        num_samples=20
    )

    # --- 4. VISUALIZE THE RESULTS ---
    low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

    plt.figure(figsize=(12, 6))

    # A. Plot the History (Black)
    plt.plot(np.arange(split_idx), train_data, color="black", label="Training History")

    # B. Plot the TRUE Future (Green Dashed) - This is what actually happened
    plt.plot(np.arange(split_idx, len(ts_data)), valid_data, color="green", linestyle="--", linewidth=2, label="True Future (Hidden)")

    # C. Plot the AI Forecast (Red) - This is what the Fine-Tuned AI predicted
    plt.plot(np.arange(split_idx, len(ts_data)), median, color="red", linewidth=2, label="Fine-Tuned Forecast")
    plt.fill_between(
        np.arange(split_idx, len(ts_data)),
        low, high, color="red", alpha=0.2, label="Uncertainty"
    )

    plt.title("The Final Test: Can the Fine-Tuned AI Predict the Cenozoic?", fontsize=14)
    plt.xlabel("Time Steps")
    plt.ylabel("Biodiversity")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    # --- 5. Calculate Chronos Accuracy ---
    forecast_median = np.quantile(forecast[0].numpy(), 0.5, axis=0)
    chronos_mse = mean_squared_error(valid_data.numpy(), forecast_median)

    print(f"--- FINAL SCOREBOARD ---")
    print(f"Chronos (Fine-Tuned Model) MSE: {chronos_mse:.4f}")

    print("\n--- HOW TO JUDGE THE WINNER ---")
    print(f"Your Fine-Tuned Chronos Error is: {chronos_mse:.4f}")
    print("Compare this to your Neural ODE visual fit.")
    print("- If Chronos MSE is < 0.05 (normalized), the 'Big Data' approach won.")
    print("- If Chronos MSE is > 0.10, the 'Physics' approach (Neural ODE) likely won.")

    # --- 6. Save Final Prediction for Report ---
    results_df = pd.DataFrame({
        'Time_Step': np.arange(len(valid_data)),
        'True_Diversity': valid_data.numpy(),
        'AI_Predicted_Diversity': forecast_median
    })
    results_df.to_csv('Final_Model_Comparison_FineTuned.csv', index=False)
    print("\n-> Results saved to 'Final_Model_Comparison_FineTuned.csv'")
else:
    print("Skipping Chronos forecasting evaluation due to missing ts_data or pipeline_ft.")


## 🎉 Conclusion
This notebook explored the Lilliput Effect using a multi-faceted approach, combining traditional machine learning with advanced deep learning techniques.

### Key Findings:
*   **Random Forest Analysis:** Identified absolute latitude as the primary driver of body size, followed by climate (d18O), suggesting strong geographic and ecological signals. Lithology also played a role, pointing to potential taphonomic influences.
*   **Lilliput Effect Visualization:** Visually demonstrated a general inverse relationship between global temperature and mean body size, supporting the Lilliput Effect, though with caveats for linearity and synchronicity.
*   **Neural ODEs:** Introduced as a powerful method for learning continuous dynamics from sparse, irregularly sampled data, highly relevant for paleontological research.
*   **Chronos-T5 Forecasting:** While providing an initial forecast of biodiversity, the fine-tuned Chronos-T5 model achieved an MSE of 13.0511. This high error suggests that for this specific paleontological time series, a "physics-based" approach (like Neural ODEs) might be more suitable than this "Big Data" model in its current fine-tuned state.

### Overall Insights:
The study highlights the complex interplay of environmental and geological factors in shaping ancient life. While machine learning can identify correlations, models capable of learning underlying dynamic processes (such as Neural ODEs) may offer deeper insights into the mechanisms driving macroevolutionary trends in sparse paleontological datasets. Further research should focus on integrating these diverse modeling approaches to gain a more comprehensive understanding of deep-time biological phenomena.
